# What is reference counting?

- We'll start with an example: let's say we run the line

```python
my_var = 10
```

- Our variable is assigned to the memory address 1000
    - So far, it's the only variable assigned to that address, so we say the **reference count is 1** for that address
    
- Now, we run the following line

```python
other_var = my_var
```

- Now, `other_var` will point to the same location in memory as `my_var` i.e. they're both aliases for the memory address 1000
    - This means that now the **reference count for the address 1000 is 2**
    
- Next, let's say we redefine `my_var` as:

```python
my_var = None
```

- Now, `my_var` no longer points to the memory address 1000
    - The only variable still pointing to that address is `other_var`
        - Therefore, the **reference count has decreased back to 1**
        
- Finally, we run the line:

```python
other_var = None
```

- Now, we've redefined both variables to point to the location of `None`
    - Neither still points to address 1000
        - Therefore, the **reference count has decreased to 0**
        
- Now that no variables point to address 1000, the information stored in the address is deleted 

____

# How can we see the reference count of a memory address?

- If we have some variable `my_var`, we can see the reference count for the memory address it corresponds to using `sys.getrefcount(my_var)`
    - When we run this, it'll create another reference to this address
        - Simply running the code increments it
        
- If we want to not worry about incrementing the reference count, we can run the following:

```python
address = id(my_var)
ctypes.c_long.from_address(address).value
```

- As we can see, we pass in the actual integer value of the address instead of the variable

____

# Example

In [4]:
import sys

In [5]:
a = [1,2,3]

- Running this code does the following:
    - Creates a list object
    - Stores the list object in memory
    - Defines variable `a` as an alias for the memory address where the list object is stored

In [6]:
id(a)

1855227387144

- This is the address of our list

In [7]:
sys.getrefcount(a)

2

- As we can see, there are two references to our memory address
    1. the actual list
    2. `sys.getrefcount(a)` added a second reference simply by executing

- Let's try using `ctypes` to get the reference count without incrementing it

In [8]:
import ctypes

- We'll create a wrapper function to make things easier

In [9]:
def ref_count(address: int):
    return ctypes.c_long.from_address(address).value

In [12]:
ref_count(id(a))

1

- As we can see, the reference count is correctly listed as 1

- **But wait a minute!**
    - When we ran `sys.refcount(a)`, it created a second reference to the memory address
        - *So why isn't the ref count still 2?*
            - When the `sys.getrefcount` finishes running, it releases its pointer to that memory address

- Now, we define `b`

In [13]:
b = a

- We double check that it has the same memory address as `a`

In [14]:
print(id(b), id(a))

1855227387144 1855227387144


- Now we get the reference count

In [15]:
ref_count(id(a))

2

- Even further

In [16]:
c = b

In [17]:
ref_count(id(a))

3

- And now, if we redefine `c`

In [18]:
c = 10
ref_count(id(a))

2

- Increments down
    - And once more

In [19]:
b = None
ref_count(id(a))

1

- Let's see what happens to the ref count for the original memory address if we redefine `a`

In [20]:
a_address = id(a)
a = None
ref_count(a_address)

1

- Shouldn't it be zero?
    - Let's try rerunning it

In [21]:
ref_count(a_address)

0

- Okay that's what we were expecting
    - But let's try it again

In [41]:
ref_count(a_address)

5

- Now it's way higher
    - *What's going on?*

- When `a` was set to `None`, the memory address becomes available for something else
    - Behind the scenes, what is stored at that address is changing once it's freed up